In [5]:
pip install tensorflow


Traceback (most recent call last):
  File "<frozen importlib._bootstrap>", line 1027, in _find_and_load
  File "<frozen importlib._bootstrap>", line 1006, in _find_and_load_unlocked
  File "<frozen importlib._bootstrap>", line 688, in _load_unlocked
  File "<frozen importlib._bootstrap_external>", line 883, in exec_module
  File "<frozen importlib._bootstrap>", line 241, in _call_with_frames_removed
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/build_env.py", line 19, in <module>
    from pip._internal.cli.spinners import open_spinner
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/spinners.py", line 9, in <module>
    from pip._internal.utils.logging import get_indentation
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/utils/logging.py", line 13, in <module>
    from pip._vendor.rich.console import (
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/rich/console.py", line 64, in <module>
    from .screen import Screen
  File "

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

In [ ]:
# Load and preprocess the dataset
def load_data(file_path):
    with open(file_path, 'r') as file:
        text = file.read().lower()
    return text

# Prepare the tokenizer and sequences
def prepare_sequences(text):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts([text])
    total_words = len(tokenizer.word_index) + 1
    input_sequences = []

    for line in text.split('\n'):
        token_list = tokenizer.texts_to_sequences([line])[0]
        for i in range(1, len(token_list)):
            n_gram_sequence = token_list[:i + 1]
            input_sequences.append(n_gram_sequence)

    max_sequence_length = max(len(x) for x in input_sequences)
    input_sequences = pad_sequences(input_sequences, maxlen=max_sequence_length, padding='pre')
    X, y = input_sequences[:, :-1], input_sequences[:, -1]
    y = tf.keras.utils.to_categorical(y, num_classes=total_words)

    return X, y, total_words


In [ ]:
# Define the model
def create_model(total_words, max_sequence_length):
    model = Sequential()
    model.add(Embedding(total_words, 100, input_length=max_sequence_length - 1))
    model.add(LSTM(150))
    model.add(Dense(total_words, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# Train the model
def train_model(model, X, y, epochs=50):
    model.fit(X, y, epochs=epochs, verbose=1)

In [ ]:
# Generate next words
def generate_next_words(model, tokenizer, seed_text, next_words=5):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_length - 1, padding='pre')
        predicted_probs = model.predict(token_list, verbose=0)
        predicted_word_index = np.argmax(predicted_probs, axis=-1)
        predicted_word = tokenizer.index_word[predicted_word_index[0]]
        seed_text += " " + predicted_word
    return seed_text

# Example usage
file_path = 'ArianaGrande.csv'  # Replace with your text file path
text_data = load_data(file_path)
X, y, total_words = prepare_sequences(text_data)
max_sequence_length = X.shape[1]

model = create_model(total_words, max_sequence_length)
train_model(model, X, y, epochs=30)

# Generate text
seed_text = "thought i'd end up "  # Replace with your seed text
print(generate_next_words(model, tokenizer, seed_text))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/30
3186/3186 ━━━━━━━━━━━━━━━━━━━━ 127s 38ms/step - accuracy: 0.0656 - loss: 6.0542
Epoch 2/30
3186/3186 ━━━━━━━━━━━━━━━━━━━━ 139s 38ms/step - accuracy: 0.2095 - loss: 4.4629
Epoch 3/30
3186/3186 ━━━━━━━━━━━━━━━━━━━━ 142s 38ms/step - accuracy: 0.3246 - loss: 3.5626
Epoch 4/30
3186/3186 ━━━━━━━━━━━━━━━━━━━━ 122s 38ms/step - accuracy: 0.4128 - loss: 2.9690
Epoch 5/30
3186/3186 ━━━━━━━━━━━━━━━━━━━━ 142s 38ms/step - accuracy: 0.4897 - loss: 2.5075
Epoch 6/30
3186/3186 ━━━━━━━━━━━━━━━━━━━━ 142s 38ms/step - accuracy: 0.5506 - loss: 2.1525
Epoch 7/30
3186/3186 ━━━━━━━━━━━━━━━━━━━━ 143s 39ms/step - accuracy: 0.6047 - loss: 1.8813
Epoch 8/30
3186/3186 ━━━━━━━━━━━━━━━━━━━━ 142s 38ms/step - accuracy: 0.6499 - loss: 1.6555
Epoch 9/30
3186/3186 ━━━━━━━━━━━━━━━━━━━━ 122s 38ms/step - accuracy: 0.6872 - loss: 1.4703
Epoch 10/30
3186/3186 ━━━━━━━━━━━━━━━━━━━━ 123s 38ms/step - accuracy: 0.7180 - loss: 1.3051
Epoch 11/30
3186/3186 ━━━━━━━━━━━━━━━━━━━━ 122s 38ms/step - accuracy: 0.7464 - loss: 1.17

NameError: name 'tokenizer' is not defined

In [4]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import numpy as np

# Load and preprocess the dataset
def load_data(file_path):
    with open(file_path, 'r') as file:
        text = file.read().lower()
    return text

# Prepare the tokenizer and sequences
def prepare_sequences(text):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts([text])
    total_words = len(tokenizer.word_index) + 1
    input_sequences = []

    for line in text.split('\n'):
        token_list = tokenizer.texts_to_sequences([line])[0]
        for i in range(1, len(token_list)):
            n_gram_sequence = token_list[:i + 1]
            input_sequences.append(n_gram_sequence)

    max_sequence_length = max(len(x) for x in input_sequences)
    input_sequences = pad_sequences(input_sequences, maxlen=max_sequence_length, padding='pre')
    X, y = input_sequences[:, :-1], input_sequences[:, -1]
    y = tf.keras.utils.to_categorical(y, num_classes=total_words)

    return X, y, total_words, tokenizer, max_sequence_length

# Define the RNN model
def create_model(total_words, max_sequence_length):
    model = Sequential()
    model.add(Embedding(total_words, 100, input_length=max_sequence_length - 1))
    model.add(SimpleRNN(150))
    model.add(Dense(total_words, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# Train the model and save training history for plotting
def train_model(model, X, y, epochs=50):
    history = model.fit(X, y, epochs=epochs, verbose=1)
    return history

# Generate next words
def generate_next_words(model, tokenizer, seed_text, max_sequence_length, next_words=5):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_length - 1, padding='pre')
        predicted_probs = model.predict(token_list, verbose=0)
        predicted_word_index = np.argmax(predicted_probs, axis=-1)
        predicted_word = tokenizer.index_word.get(predicted_word_index[0], '')
        seed_text += " " + predicted_word
    return seed_text

# Plot training history
def plot_training_history(history):
    plt.plot(history.history['loss'], label='Loss')
    plt.plot(history.history['accuracy'], label='Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Metrics')
    plt.legend()
    plt.show()

# Visualize embeddings with t-SNE clustering
def plot_word_embeddings(model, tokenizer):
    embeddings = model.layers[0].get_weights()[0]  # Get the embedding layer weights
    words = list(tokenizer.word_index.keys())
    word_embeddings = embeddings[:len(words)]  # Limit embeddings to words in tokenizer

    tsne = TSNE(n_components=2, random_state=0, perplexity=5)
    tsne_embeddings = tsne.fit_transform(word_embeddings)

    plt.figure(figsize=(10, 10))
    for i, word in enumerate(words):
        plt.scatter(tsne_embeddings[i, 0], tsne_embeddings[i, 1])
        plt.annotate(word, (tsne_embeddings[i, 0], tsne_embeddings[i, 1]), fontsize=9)
    plt.show()

# Example usage
file_path = 'ArianaGrande.csv'  # Replace with your text file path
text_data = load_data(file_path)
X, y, total_words, tokenizer, max_sequence_length = prepare_sequences(text_data)

model = create_model(total_words, max_sequence_length)
history = train_model(model, X, y, epochs=20)

# Plot training metrics
plot_training_history(history)

# Plot word embeddings with t-SNE
plot_word_embeddings(model, tokenizer)

# Generate text
seed_text = "thought i'd end up "
print(generate_next_words(model, tokenizer, seed_text, max_sequence_length))


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/20
 482/3186 ━━━━━━━━━━━━━━━━━━━━ 21:33 478ms/step - accuracy: 0.0111 - loss: 8.1459

KeyboardInterrupt: 